In [8]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Initialize counter and state
counter = 0
state = 'none'  # 'none', 'cat', 'cow'

# Define a function to detect cat and cow poses
def detect_pose(landmarks):
    cat_pose = False
    cow_pose = False

    # Extract relevant landmarks
    nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
    shoulder_left = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    shoulder_right = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    hip_left = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    hip_right = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    knee_left = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    knee_right = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

    # Calculate angles and positions
    shoulder_hip_angle_left = calculate_angle(shoulder_left, hip_left, knee_left)
    shoulder_hip_angle_right = calculate_angle(shoulder_right, hip_right, knee_right)
    head_angle_left = calculate_angle(nose, shoulder_left, hip_left)
    head_angle_right = calculate_angle(nose, shoulder_right, hip_right)

    # Define thresholds for cat and cow poses
    cat_pose = (shoulder_hip_angle_left < 120 and shoulder_hip_angle_right < 120 and (head_angle_left < 45 or head_angle_right < 45))
    cow_pose = (shoulder_hip_angle_left > 150 and shoulder_hip_angle_right > 150 and (head_angle_left > 45 or head_angle_right > 45))

    return cat_pose, cow_pose

def calculate_angle(a, b, c):
    a = np.array([a.x, a.y])  # First point
    b = np.array([b.x, b.y])  # Mid point
    c = np.array([c.x, c.y])  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to find the pose
    results = pose.process(rgb_frame)

    # Draw the pose landmarks on the frame
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Detect cat and cow poses
        cat_pose, cow_pose = detect_pose(results.pose_landmarks.landmark)

        # State machine to count the sequence of poses
        if state == 'none':
            if cat_pose:
                state = 'cat'
            elif cow_pose:
                state = 'cow'
        elif state == 'cat':
            if cow_pose:
                counter += 1
                state = 'none'
        elif state == 'cow':
            if cat_pose:
                counter += 1
                state = 'none'

        # Display the pose detection result
        if cat_pose:
            cv2.putText(frame, 'Cat Pose', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif cow_pose:
            cv2.putText(frame, 'Cow Pose', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame, 'No Pose Detected', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Display the counter
    cv2.putText(frame, f'Counter: {counter}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the frame
    cv2.imshow('Pose Detection', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
